## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(64,kernel_size=(3,3),input_shape=(32,32,3),padding='same',activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.2)) 

#卷積組合
classifier.add(Convolution2D(64,kernel_size=(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.2)) 

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(2,2),padding='same',activation='relu'))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.2)) 

#flatten
classifier.add(Flatten())
#classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(512,activation='relu')) #output_dim=100,activation=relu
classifier.add(Dropout(0.2)) 

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

classifier.summary()

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 64)        256       
_______________

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [5]:
train_history = classifier.fit(x_train,y_train,validation_split=0.2,batch_size=100,epochs=50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 9s 222us/step - loss: 1.5307 - acc: 0.4604 - val_loss: 1.2814 - val_acc: 0.5642
Epoch 2/50
40000/40000 [==============================] - 6s 141us/step - loss: 1.1472 - acc: 0.5901 - val_loss: 0.9873 - val_acc: 0.6475
Epoch 3/50
40000/40000 [==============================] - 5s 137us/step - loss: 0.9915 - acc: 0.6460 - val_loss: 0.9180 - val_acc: 0.6754
Epoch 4/50
40000/40000 [==============================] - 6s 138us/step - loss: 0.8850 - acc: 0.6873 - val_loss: 0.8270 - val_acc: 0.7137
Epoch 5/50
40000/40000 [==============================] - 6s 139us/step - loss: 0.8103 - acc: 0.7106 - val_loss: 0.8093 - val_acc: 0.7205
Epoch 6/50
40000/40000 [==============================] - 6s 145us/step - loss: 0.7549 - acc: 0.7312 - val_loss: 0.7446 - val_acc: 0.7446
Epoch 7/50
40000/40000 [==

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.6619821e-04, 1.9492604e-15, 9.2426002e-01, 1.9748416e-02,
        1.4047576e-04, 1.1430731e-08, 5.5621207e-02, 3.4345438e-13,
        6.3637330e-05, 6.7148977e-13]], dtype=float32)

In [7]:
scores = classifier.evaluate(x_test, y_test)
print('accuracy=', scores[1])

10000/10000 [==============================] - 1s 110us/step
accuracy= 0.7914
